# 1. Accessing Twitter API
### Elisabeth Earley

In [1]:
#Import required packages
import tweepy
import json
import csv
import pandas as pd
import datetime
import time

In [85]:
print("Last run: " + str(pd.to_datetime('today')))

Last run: 2018-04-23 00:00:00


In [2]:
#authorize my app to begin using the API!
access_token = 'XXX'
access_token_secret = 'XXX'
consumer_key = 'XXX'
consumer_secret = 'XXX'


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

API = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True) #waiting on rate limit automatically sleeps
                                                                                #the system (15 mins) when rate limit has been 
                                                                                #reached.

In [4]:
#create dictionary of Colombian elites and their real name + username
elites = {'Alvaro Uribe Velez':'@AlvaroUribeVel', 'Sergio Fajardo':'@sergio_fajardo', \
'Gustavo Petro':'@petrogustavo', 'Ivan Duque Marquez':'@IvanDuque', 'German Vargas Lleras':'@German_Vargas', \
'Juan Manuel Santos':'@juanmansantos', 'Humberto de la Calle':'@DeLaCalleHum'}
names = elites.keys()  #just their real names
handles = elites.values() #just their twitter usernames

In [5]:
#create a profile for each elite that contains all their user information

profiles = {}
for user in elites:
    profiles[user] = API.get_user(elites[user])

# A) Elite "followers"
[Still under construction...each candidate can hve up to 5 million followers. I can only
grab 5000 ID's at a time and can only use those ID's to pull profiles 900 at at time. It would take 13 days to process 1 million followers. Pulling profiles directly can only be done 200 at a time which would take 52 days for 1 million followers.]
### Saving for future use!

In [6]:
follow_id_dict = {}
for user in elites:
    follow_id_dict[user] = API.followers_ids(elites[user], cursor=-1, count=5000)
    

Example of ID list. From the above loop, each candidate has 5000 follower IDs in their list.

In [47]:
print((follow_id_dict['Gustavo Petro'][0][:20]))

[988687871673622528, 988687996529598465, 988687296227692544, 988686975950639104, 139164299, 986464756117725185, 804570031, 988526901089177600, 443798852, 988685213768265728, 742544146599170048, 519745004, 2364116309, 943322239251906560, 113583414, 988680121124052992, 1058936515, 988677731901636608, 184560506, 988672139120214017]


# B) Elite "followings" (aka "friends")
1. Gather ID #'s for the accounts that each elite is following 
2. Use ID's to quickly pull user information
3. Save id, name, screen_name, location, created_at, friends_count, followers_count into csv for each elite

In [7]:
friend_id_dict = {}
for user in elites:
    friend_id_dict[user] = API.friends_ids(elites[user])
    
#Create a dictionary for the total list of 'friends' IDs, where each elite has their own list.

In [ ]:
#These are the numbers of IDs that we should be pulling. More for some than others.
for name in names:
    print("Total friends for: " + name)
    print(profiles[name].friends_count)
    print('\n')
    

### Create function that downloads specific variables from each 'friend's' profile, writing the data into a csv file
**Expected time: 2.64 hours**

1. Create empty dictionary to store an empty list for every elite.
2. Create an empty csv ('friends.csv') in sublime to avoid the meta-data created by opening a new file in excel.
3. Using empty csv and encoding in utf-8 to avoid the loop breaking due to emojis, create columns for the variables i'm interested in taking from each 'friend'
4. Loops through the ID numbers for each elite's friends list and grabs the 'friend' profile
5. Allow the loop to append 'friend' profiles to the respective empty list within the main dictionary (userlist)
6. Create a dictionary holding each column's information (row_dict)
7. Allow the csv to be written using the row_dict data 

In [8]:
userlist = {}
with open('friends.csv', 'w', encoding='utf-8') as csvfile:
    fieldnames = ['elite', 'id', 'name', 'screen_name', 'location', 'created_at', 'friends_count', 'followers_count']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for name in list(elites.keys()):
        userlist[name] = []
        for id_no in friend_id_dict[name]:
            print(name, id_no)
            friend_data = API.get_user(id_no)
            userlist[name].append(friend_data)
            row_dict = {}
            row_dict['elite'] = name
            row_dict['id'] = id_no
            row_dict['name'] = friend_data.name
            row_dict['screen_name'] = friend_data.screen_name
            row_dict['location'] = friend_data.location
            row_dict['created_at'] = friend_data.created_at
            row_dict['friends_count'] = friend_data.friends_count
            row_dict['followers_count'] = friend_data.followers_count
            writer.writerow(row_dict)
        

Alvaro Uribe Velez 959979685844324352
Alvaro Uribe Velez 188794797
Alvaro Uribe Velez 444864072
Alvaro Uribe Velez 235307099
Alvaro Uribe Velez 924605888


Alvaro Uribe Velez 138235377
Alvaro Uribe Velez 48105688
Alvaro Uribe Velez 702228317202751488
Alvaro Uribe Velez 189305014
Alvaro Uribe Velez 540894368


Alvaro Uribe Velez 58977300
Alvaro Uribe Velez 30861738
Alvaro Uribe Velez 275669169
Alvaro Uribe Velez 314907522
Alvaro Uribe Velez 2953203339


Alvaro Uribe Velez 2196442651
Alvaro Uribe Velez 209134275
Alvaro Uribe Velez 4573732999
Alvaro Uribe Velez 632417584
Alvaro Uribe Velez 580993736


Rate limit reached. Sleeping for: 718


...

Ivan Duque Marquez 854291
Ivan Duque Marquez 295779307
Ivan Duque Marquez 9572932


German Vargas Lleras 300815030
German Vargas Lleras 2400080066
German Vargas Lleras 231803432
German Vargas Lleras 14050583
German Vargas Lleras 135321834


Rate limit reached. Sleeping for: 718


Juan Manuel Santos 286441121
Juan Manuel Santos 1132257020
Juan Manuel Santos 58244743


Humberto de la Calle 383579991
Humberto de la Calle 102171561
Humberto de la Calle 147705809
Humberto de la Calle 101736223


In [43]:
#quick unicode data clean
#I want to strip all accents from letters FIRST to avoid unicode errors going into R
#locations were being interpreted into R as "PopayÃ¡n, Colombia", etc. 

import unidecode
with open('friends_cleaned.csv', 'w', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    with open('friends.csv', 'r', encoding='utf-8') as friends:
        reader = csv.reader(friends, delimiter=',', quotechar='"')   #quotechar is needed so that the commas in the string don't
        for row in reader:                                           #split the location into two. (e.g. "Bogota, Colombia")
            if len(row) > 0:  #only run this function on non-missing data
                location = row[4]
                cleaned_location = unidecode.unidecode(location).strip('"') #first removes accents, then removes " " in the csv
                print(location, cleaned_location) #make sure it's working right by
                new_row = list(row)               #printing old string and new string
                new_row[4] = cleaned_location
                writer.writerow(new_row)
                
            
    


location location
Barbosa, Colombia Barbosa, Colombia
Colombia, Panamá y USA Colombia, Panama y USA
Universidad de los Andes Universidad de los Andes
Bogotá, DC, Colombia Bogota, DC, Colombia
Bogotá, D.C., Colombia Bogota, D.C., Colombia
Armenia, Colombia Armenia, Colombia
 
Bogotá, D.C. Colombia Bogota, D.C. Colombia
 
 
 
 
Medellín Medellin
Caracas, Venezuela Caracas, Venezuela
Aguascalientes, México Aguascalientes, Mexico
 
Medellín, Colombia Medellin, Colombia
Bogotá, D.C., Colombia Bogota, D.C., Colombia
Medellín, Colombia Medellin, Colombia
Valle del Cauca, Colombia Valle del Cauca, Colombia
 
Ibagué, Tolima Ibague, Tolima


New york-Italy-Mexico New york-Italy-Mexico
Medellín, Colombia Medellin, Colombia
 
Cali, Valle del Cauca Cali, Valle del Cauca
Guatape, Colombia Guatape, Colombia
 
 
Colombia Colombia
 
 
BOGOTA D.C. - COLOMBIA BOGOTA D.C. - COLOMBIA
Bogotá, Colombia Bogota, Colombia


### ...Time to move to data cleaning and visualization in R
(After waiting 3 hours for the data collecting loop to run and write the csv all the way through! I ran mine overnight.)

# C) Download tweets from time-windows
### Saving for future use!

In [87]:
startDate_1 = datetime.datetime(2014, 5, 1, 0, 0, 0) #elections
endDate_1 = datetime.datetime(2014, 8, 1, 0, 0, 0)

startDate_2 = datetime.datetime(2016, 9, 1, 0, 0, 0) #Plebiscite vote on ratification
endDate_2 = datetime.datetime(2016, 12, 31, 0, 0, 0)

startDate_3 = datetime.datetime(2018, 3, 1, 0, 0, 0) #current elections
endDate_3 = datetime.datetime(2018, 7, 1, 0, 0, 0)


In [ ]:
# for name in elites:
#     tweepy.Cursor(API.user_timeline, elites[name])


In [55]:
#test_tweets = API.user_timeline(screen_name='IvanDuque', since_id=981697880766468097, count=20, tweet_mode='extended')

In [62]:
# print("Recent tweets:\n")
# for tweet in test_tweets:
#     print(tweet.full_text)
#     print()

Recent tweets:

Está noche estaremos nuevamente en #PreguntaYamid en el canal @elunodetodos @CMILANOTICIA Acompáñanos! #DuqueEsElQueEs https://t.co/PbeCKQasyr

RT @CMILANOTICIA: Hoy desde las 10:00 p. m. no te pierdas en #PreguntaYamid  la segunda parte de la entrevista al candidato presidencial @I…

La economía no se recupera a base de impuestos, se recupera con un gobierno austero, que elimine el derroche y los gastos innecesarios, que piense en el ciudadano y permita mejorar la inversión y generar empleo formal #HotelElPradro #DuqueEsElQueEs https://t.co/0MIUqA6hw7

Aquí está el segundo carnaval más grande de América Latina, donde el folclor y la cultura unen a todo un país. Barranquilla nos demuestra la importancia de apostarle a la economía naranja y a las industria creativas #HotelElPradro #DuqueEsElQueEs https://t.co/t4WLolIs33

Vamos a llegar a la presidencia a defender la legalidad, a luchar contra la impunidad, contra la inseguridad, contra la prostitución infantil. En defens